In [1]:
kernel.silent(true)

In [2]:
import coursierapi.MavenRepository
interp.repositories() ++= Seq(MavenRepository.of("https://jitpack.io"))

In [3]:
import $ivy.`com.github.propi:rdfrules:1.5.0`
import collection._
import org.apache.jena.riot.Lang

import com.github.propi.rdfrules.data._
import com.github.propi.rdfrules.algorithm.amie._
import com.github.propi.rdfrules.algorithm.dbscan._
import com.github.propi.rdfrules.utils._
import com.github.propi.rdfrules.index._
import com.github.propi.rdfrules.rule._
import com.github.propi.rdfrules.ruleset._

In [4]:
import reflect.io._, Path._
import reflect.io._
import Path._

In [5]:
val uri = (value: String) => TripleItem.Uri(value)
val qbDataSet = "http://purl.org/linked-data/cube#dataSet";

In [6]:
val graphs = "../data/salaries".toDirectory.files.map(_.path).filter(g => g matches """.*\.ttl""")

In [7]:
var salariesCubesMap: Map[String, Graph] = Map()
graphs.foreach(g => {
    val name = g.replaceAll("\\.ttl", "").replaceAll("^.*/", "")
    salariesCubesMap = salariesCubesMap + (name -> Graph(g))
})

In [8]:
import eu.easyminer.discretization.impl.Interval
import eu.easyminer.discretization.impl.IntervalBound._
val rounded = (value: Double, scale: Integer) => BigDecimal(value).setScale(scale, BigDecimal.RoundingMode.HALF_UP).toDouble
val intervalToString = (i: Interval) => "<"+rounded(i.minValue.value,2)+ "__"+rounded(i.maxValue.value,2)+")"

In [9]:
val averageSalary = TripleItem.Uri("http://data.czso.cz/ontology/prumernaMzda")
val medianSalary = TripleItem.Uri("http://data.czso.cz/ontology/medianMzdy")
val measureUris = Array(averageSalary,medianSalary)

In [10]:
val equiFrequent = (intervalCount: Int) => DiscretizationTask.Equifrequency(intervalCount)

val discretizeEquifrequent = (graph: Graph, intervalsCount: Int, measure: TripleItem.Uri) => {
    graph.filter(t => t.predicate == measure)
         .discretize(equiFrequent(intervalsCount))(quad => quad.triple.predicate == measure)
}

val discretizeEquifrequentAndGetIntervals = (graph: Graph, intervalsCount: Int, measure: TripleItem.Uri) => {
    graph.filter(t => t.predicate == measure)
         .discretizeAndGetIntervals(equiFrequent(intervalsCount))(quad => quad.triple.predicate == measure)
}

In [11]:
import eu.easyminer.discretization.impl.Interval

val renameInterval = (t: Triple, intervals: IndexedSeq[Interval], suffix: String) => {
    var result = t
    intervals.foreach(interval => {
        val _interval = t.`object`.asInstanceOf[TripleItem.Interval]
        if (_interval.interval == interval) {
            result = t.copy(`object` = TripleItem.Uri(_interval.toString+"_"+suffix+"_"+(intervals.indexOf(interval)+1)+"/"+intervals.size) )
        }
    })
    result
}

val discretize = (graph: Graph, name: String, intervalCounts: Array[Int]) => {
    var result = Dataset() + 
    graph.filter(t => !measureUris.contains(t.predicate))
    .map(t => if (t.predicate.hasSameUriAs(qbDataSet)) t.copy(`object` = uri(name)) else t)
    
        measureUris.foreach(measure => {
            intervalCounts.foreach(intervalCount => {
                val intervals = discretizeEquifrequentAndGetIntervals(graph, intervalCount, measure)
                val measures = discretizeEquifrequent(graph, intervalCount, measure)
                result = result + measures.map(t => {renameInterval(t, intervals, "ef"+intervalCount)})
            })
    })
    result
}

In [12]:
val intervalCounts = Array(2, 3, 5, 7)
var dataset = Dataset()
salariesCubesMap
.map(t => discretize(t._2,t._1,intervalCounts))
.foreach(d => dataset = dataset + d)

In [13]:
salariesCubesMap
.foreach(d => 
         discretize(d._2,d._1,intervalCounts)
         .cache("../cache/salaries/"+d._1+".cache")
         .export("../exports/salaries/"+d._1+".ttl")
)